In [1]:
import numpy as np
import itertools as it

In [5]:
def base(M):
    # calculate the image of the base under a matrix M
    
    # make a set of all combinations of the last n-1 columns (these correspond to Z_2,...,Z_n)
    s = list([M[0:2*n, i] for i in range(n+1, 2*n)])
    powerset = it.chain.from_iterable(it.combinations(s, r) for r in range(1, len(s)+1)) 
    
    base_map = set()
    for i in powerset:
        v = vector(sum(i))     # calculate the sum of the elements of each combination (e.g IZZ = IZI + IIZ)
        v.set_immutable()
        base_map.add(v)
    return frozenset(base_map)

In [36]:
# number of qubits (n > 1)
n = 5

# calculate number of cosets
prod = 1
for i in range(1,n+1):
    prod = prod * (2**i + 1)
index = 1/3 * (2**n - 1) * prod

# load symplectic group
Cn = Sp(2*n, GF(2))

# calculate matrix needed for transformation from 'sage symplectic' to 'literature symplectic'
anti_identity = matrix.identity(n)
for i in range(0, (n/2).ceil()):
    anti_identity.swap_rows(i, n-i-1) 
T = block_matrix(GF(2), [[matrix.identity(n), zero_matrix(n,n)], [zero_matrix(n,n), anti_identity]], subdivide = False)

In [37]:
transversal_inv = {}

while len(transversal_inv)<index:
    print('%d/%d'%(len(transversal_inv)+1,index), end='\r')
    # generate random element of symplectic group
    M = T * Cn.random_element() * T
    
    # calculating M inverse
    A = M[0:n, 0:n]
    B = M[0:n, n:2*n]
    C = M[n:2*n, 0:n]
    D = M[n:2*n, n:2*n]
    M_inv = block_matrix([[D.transpose(),-B.transpose()],[-C.transpose(),A.transpose()]], subdivide=False)
    M_basecol = base(M_inv)
    
    # check whether coset is already in transversal
    if M_basecol not in transversal_inv: 
        transversal_inv[M_basecol] = M_inv


KeyboardInterrupt: 

In [ ]:
# save transversal
save(transversal_inv, '%d_transversal_inv.sobj'%n)